# Sistema de recomendación basado en contenido 

Este tipo de recomendadadores se centran en la recoemndación personalizada: en este caso a cada usuario se le recomienda contenido diferente, buscando siempre que el contenido ofrecido se pueda ajustar a los gustos del usuario. En estos casos es dificil que dos personas reciban la misma recomendación.


Los sistemas de recomendación personalizados existen tres grandes enfoques que se pueden aplicar: basarse en contenido, basarse en usuarios, y un sistema híbrido. Veamos en qué consiste cada uno de ellos.

Como su nombre indica un sistema de recomendación basado en contenido utiliza las características del contenido para hacer las recomendaciones. En otras palabras, un sistema de recomendación basado en contenido recomendará contenidos similares a los contenidos que le han gustado al usuario.

Supongamos que me gusta mucho los super heroes y llevo toda la semana viendo peliculas de Marvel. Un sistema de recoemndacion me recomendaria peliculas o series de ciencia ficcion, en las que apareciera 	Robert Downey Jr. y dirigidas  por los hermanos Russo, que ademas fueran del mismo ano...etc

La clave de los sistemas de recomendacion basados en el contenido es definir las caracteristicas que se usaran para encontrar productos parecidos. En este caso lo mas importante para un buen sistema de recomendacion serian actor o actores principales , director o directores , categoria de pelicula...etc



## Pros y contras del sistema de recomendación basado en contenido.
- Pros del sistema de recomendación basado en contenido
    - Permiten realizar predicciones desde el momento en el que un usuario consume un producto. Es decir, el sistema no sufre del problema conocido como cold-start.
    - Es un sistema adaptativo: si el usuario cambia sus gustos, el sistema se dará cuenta de esos cambios.
    - Recomienda productos poco populares.
    - Fácil de escalar.
    

- Contras del sistema de recomendación basado en contenido
    - El contenido recomendado siempre es parecido. Es decir, si una persona consume peliculas de acción es dificil que se le recomienden productos de otra categoría.


## Ejemplo 
Vamos a usar el dataset IMDB, con información sobre más de 1.000 peliculas y series valoradas en IMDB. Puedes descargarte el dataset desde [aquí](https://www.kaggle.com/datasets/harshitshankhdhar/imdb-dataset-of-top-1000-movies-and-tv-shows).



In [1]:
# %%
# libreries


import pandas as pd # data processing 


# sklearn libraries for preprocessing and model building 
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline

# calculo de matriz de similitudes
from sklearn.neighbors import NearestNeighbors




In [2]:
imdb = pd.read_csv('data/imdb_top_1000.csv')
imdb.head()

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


Este set de datos contiene muchas variables que nos perimten caracterizar las peliculas. 


## Preparacion de datos

Para este ejemplo seleccionaremos las caracterisaticas de las peliculas (genero, director, actores ... etc) obviando las descripciones para evitar ruido en el modelo. 

In [4]:
# Filter data
keep_cols = ['Genre', 'Director', 'Star1', 'Star2']
imdb_filtered = imdb[keep_cols]
imdb_filtered

,Genre,Director,Star1,Star2
0,Drama,Frank Darabont,Tim Robbins,Morgan Freeman
1,"Crime, Drama",Francis Ford Coppola,Marlon Brando,Al Pacino
2,"Action, Crime, Drama",Christopher Nolan,Christian Bale,Heath Ledger
3,"Crime, Drama",Francis Ford Coppola,Al Pacino,Robert De Niro
4,"Crime, Drama",Sidney Lumet,Henry Fonda,Lee J. Cobb
...,...,...,...,...
995,"Comedy, Drama, Romance",Blake Edwards,Audrey Hepburn,George Peppard
996,"Drama, Western",George Stevens,Elizabeth Taylor,Rock Hudson
997,"Drama, Romance, War",Fred Zinnemann,Burt Lancaster,Montgomery Clift
998,"Drama, War",Alfred Hitchcock,Tallulah Bankhead,John Hodiak


`Pipeline` es una clase que nos permite crear un flujo de procesamiento de datos. `StandardScaler` nos permite estandarizar las variables numericas para que tengan media 0 y desviacion estandar 1 y el atributo 'scaler' es el nombre que le damos a la transformacion

In [5]:
# %%
# Create pipeline for numerical variables
numeric_pipe = Pipeline([('scaler', StandardScaler())])

Teniendo en cuenta que una pelicula puede pertenecer a varias etiquetas de generos utilizarmeos la funcion `OneHotEncoder` de la libreria `Sklearn` para "dumificar esas variables".  


In [6]:
categorical_pipe = Pipeline([('encoder', OneHotEncoder(drop = 'first'))])

`ColumnTransformer` es una clase que nos permite aplicar diferentes transformaciones a diferentes columnas de un dataframe. `numeric` es el nombre que le damos a la transformacion, `numeric_pipe` es el pipeline que creamos anteriormente, `imdb_filtered._get_numeric_data().columns.tolist()` es una lista con los nombres de las columnas numericas. 

`categoric` es el nombre que le damos a la transformacion, categorical_pipe es el pipeline que creamos anteriormente, `imdb_filtered.select_dtypes('object').columns.tolist()` es una lista con los nombres de las columnas categoricas

In [7]:
col_transf = ColumnTransformer([ 
                                ('numeric', numeric_pipe, imdb_filtered._get_numeric_data().columns.tolist()), 
                                ('categoric', categorical_pipe, imdb_filtered.select_dtypes('object').columns.tolist())  
                                ])

Aplicamos las transformaciones a las columnas del dataframe mediante `.fit`

In [8]:
col_transf_fit = col_transf.fit(imdb_filtered)

`.transform` es un metodo de la clase ColumnTransformer que nos permite aplicar las transformaciones a las columnas del dataframe

In [9]:
imdb_filtered_transf = col_transf_fit.transform(imdb_filtered)
imdb_filtered_transf

<1000x2247 sparse matrix of type '<class 'numpy.float64'>'
	with 3986 stored elements in Compressed Sparse Row format>

Con esto ya tenemos los datos preparados, ahora veamos cómo crear nuestro sistema de recomendación basado en contenido.

## Generacion de sistema de recomendacion 

El sistema se basa en dos pasos:

    - Encontrar la similitud de una película que le ha gustado al usuario respecto al resto de películas.
    - Seleccionar las N peliculas que más se parezcan a la película que le ha gustado al usuario.

Este proceso se parece mucho al algoritmo kNN. Sin embargo, hay que tener en cuenta que contamos con una matriz hueca. Las matrices huecas se caracterizan por tener un gran tamano en las que la mayor parte de sus elementos son cero. Poe ese motivo no podemos aplicar cualquier tipo de distancia, sino que tendremos que aplicar una distancia como la distancia del coseno.

Para montar el sistema de recomendación basado en contenido vamos a calcular la matriz de similitudes entre las diferentes películas. Para ello vamos a usar la función `NearestNeighbor` fijando como el timpo de distancia `cosine`

In [10]:

# numero de vecinos a considerar para el calculo de la similitud entre peliculas 
n_neighbors=5 


nneighbors = NearestNeighbors(n_neighbors = n_neighbors, # numero de vecinos a considerar para el calculo de la similitud entre peliculas 
                              metric = 'cosine'          # cosin sirve para medir la similitud entre dos vectores teniendo en cuenta el angulo entre ellos 
                              ).fit(imdb_filtered_transf) # fit es un metodo de la clase NearestNeighbors que nos permite aplicar las transformaciones a las columnas del dataframe

Ya tendríamos nuestro recomendador entrenado. Ahora solo falta pasar una nueva película para hacer recomendaciones. Veamos pues qué recomienda el sistema a una persona que ve la película The Godfather:



In [11]:
# Si queremos ver peliculas similares a "The Godfather"
pelicula = "The Dark Knight"
# extraer el indice de la pelicula "The Godfather"
pelicula_index= imdb[imdb['Series_Title'] == pelicula].index
pelicula_index = pelicula_index[0]
pelicula_index

2

Calculamos la distancia entre la pelicula y sus vecinos mas cercanos (`dif`) y luego obtenemos  los indices de las peliculas mas cercanas a la pelicula que estamos analizando( `ind`)


In [12]:
# hacer prediciones con el modelo
dif, ind = nneighbors.kneighbors(imdb_filtered_transf[pelicula_index]) 

In [13]:
# cual es el valor maximo que puede tomar la distancia entre dos peliculas 
dif

array([[0.        , 0.42264973, 0.42264973, 0.5       , 0.75      ]])

In [14]:
print("Queremos ver peliculas similares a: ", imdb.loc[pelicula_index, 'Series_Title'])
print("="*80)
imdb.loc[ind[0][0], :]

Queremos ver peliculas similares a:  The Dark Knight


Poster_Link      https://m.media-amazon.com/images/M/MV5BMTMxNT...
Series_Title                                       The Dark Knight
Released_Year                                                 2008
Certificate                                                     UA
Runtime                                                    152 min
Genre                                         Action, Crime, Drama
IMDB_Rating                                                    9.0
Overview         When the menace known as the Joker wreaks havo...
Meta_score                                                    84.0
Director                                         Christopher Nolan
Star1                                               Christian Bale
Star2                                                 Heath Ledger
Star3                                                Aaron Eckhart
Star4                                                Michael Caine
No_of_Votes                                                230

In [15]:
print("Peliculas recomendadas")
print("="*80)
imdb.loc[ind[0][1:], :]

Peliculas recomendadas


,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
155,https://m.media-amazon.com/images/M/MV5BOTY4Yj...,Batman Begins,2005,UA,140 min,"Action, Adventure",8.2,"After training with his mentor, Batman begins ...",70.0,Christopher Nolan,Christian Bale,Michael Caine,Ken Watanabe,Liam Neeson,1308302,"206,852,432"
63,https://m.media-amazon.com/images/M/MV5BMTk4OD...,The Dark Knight Rises,2012,UA,164 min,"Action, Adventure",8.4,Eight years after the Joker's reign of anarchy...,78.0,Christopher Nolan,Christian Bale,Tom Hardy,Anne Hathaway,Gary Oldman,1516346,"448,139,099"
36,https://m.media-amazon.com/images/M/MV5BMjA4ND...,The Prestige,2006,U,130 min,"Drama, Mystery, Sci-Fi",8.5,"After a tragic accident, two stage magicians e...",66.0,Christopher Nolan,Christian Bale,Hugh Jackman,Scarlett Johansson,Michael Caine,1190259,"53,089,891"
345,https://m.media-amazon.com/images/M/MV5BMjEzMz...,Tropa de Elite 2: O Inimigo Agora é Outro,2010,NaN,115 min,"Action, Crime, Drama",8.0,"After a prison riot, former-Captain Nascimento...",71.0,José Padilha,Wagner Moura,Irandhir Santos,André Ramiro,Milhem Cortaz,79200,"100,119"
